<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Function-Approximation" data-toc-modified-id="Function-Approximation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Function Approximation</a></span><ul class="toc-item"><li><span><a href="#Interpolation-principles" data-toc-modified-id="Interpolation-principles-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Interpolation principles</a></span></li><li><span><a href="#Polynomial-interpolation" data-toc-modified-id="Polynomial-interpolation-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Polynomial interpolation</a></span></li></ul></li></ul></div>

# Function Approximation    

## Interpolation principles 

 **Developing an interpolation scheme**  
 
 Interpolation refers to a situation when we know the values of $\mathcal{f}$ at some discrete points in its domain and from this we are trying to infer how $\mathcal{f}$ looks at points between the points where $\mathcal{f}$ is known. In *Function approximation* problems interpolation is used to choose an approximant $\hat{f}$ that matches $\mathcal{f}$ at the known evaluation points. 

**Step 1**: let $\hat{f}$ be an approximating function for true $\mathcal{f}$. Choose a family of approximating functions for $\hat{f}$. The chapter considers $\hat{f}$ that can be written as a linear combination of *n* independent *basis functions $\phi_1$, $\phi_2$, ... $\phi_n$*   
$$\hat{f}(x)  = \sum_{j=1}^n \phi_j(x)c_j = \phi(x)c$$    

Goal: estimate the "weights", i.e. coefficient vector $\mathcal{c}$, for the approximation to work.   

What does $\mathcal{n}$ mean? $\mathcal{n}$ refers to the ***interpolation order*** . For example, $p(x) = x^0$ is interpolation of order 1, $p(x) = x^0 + x$ is interpolation of order 2. Generally, $n^{th}$-degree interpolation looks like this: $p(x) = c_1x^0 + c_2x + c_3x^2 + ... +c_nx^(n-1)$. 

**Step 2**: specify what properties we want $\hat{f}$ to replicate from $\mathcal{f}$.       

 - for example, we may want $\hat{f}$ to match the values of $\mathcal{f}$ at selected ***interpolation nodes*** $\mathcal{x}_1$, $\mathcal{x}_2$, ..., $\mathcal{x}_n$.   
 
 The estimation of coefficients reduces to solving a system of linear equations:   
 
 $$\sum_{j=1}^n \phi_j(x)c_j =  \mathcal{f}(x_i) = \mathcal{y}_i$$    
 
 or,    
 
 $$\Phi c = y $$   

In [3]:
import sympy as sym  

In [70]:
def interpolation(f, psi, nodes):
    
    """we interpolate true f at selected nodes with a polynomial that can be represented
    as a linear combinations of psi-s""" 
    
    N = len(psi) - 1 # degree of the polynomial
    PSI = sym.zeros(N+1, N+1) # square matrix, 
    """matrix size corresponds to the interpolation order - number of interpoltion nodes"""
    b = sym.zeros(N+1, 1) # original function f 
    psi_sym = psi  # save symbolic expression
    # Turn psi and f into Python functions
    x = sym.Symbol('x')
    psi = [sym.lambdify([x], psi[i]) for i in range(N+1)] # evaluating the psi-a at the nodes
    f = sym.lambdify([x], f)
    for i in range(N+1):
        for j in range(N+1):
            PSI[i,j] = psi[j](nodes[i])
        b[i,0] = f(nodes[i])
    c = PSI.LUsolve(b)
    # c is a sympy Matrix object, turn to list
    c = [sym.simplify(c[i,0]) for i in range(c.shape[0])]
    u = [sym.simplify(sum(a*b for a, b in zip(c, psi_sym)))]
    return u,c 

Now we can use the interpolation function to approximate a parabola $f(x) = 10(x-1)^2 -1$ by a linear function on interval $[1,2]$ using two interpolation nodes: 

In [71]:
x = sym.Symbol('x')
f = 10*(x-1)**2 - 1
psi = [1, x]
Omega = [1, 2]
points = [1 + sym.Rational(1,3), 1 + sym.Rational(2,3)]
u,c = interpolation(f, psi, points)   

In [72]:
c

[-119/9, 10]

In [73]:
u

[10*x - 119/9]

In [74]:
type(f)

sympy.core.add.Add

 **A few words about matrix $\Phi$**   
 $\Phi$ has to be non-singular for the interpolation scheme to be well-defined   
 $\phi_ij$ = $\phi_j(x_i)$   is a typical element of $\Phi$. $\Phi$ is a square matrix whose elements are values of each basis function evaluated at each node.    
 
 
 
 - depending on the context of the problem in question, we may want the first derivates of $\hat{f}$ to match the first derivates of  $\mathcal{f}$ at selected interpolation nodes $\mathcal{x}_1^\prime$, $\mathcal{x}_2^\prime$, ..., $\mathcal{x}_n^\prime$    
 
 $$\sum_{j=1}^n \phi_j^\prime(x^\prime) c_j =  \mathcal{f}^\prime(x_i)^\prime $$     
 
 >While developing interpolation scheme we should choose good interpolation nodes and basis functions. What does this imply?   
      First, the approximant should be able to produce an accurate approximation of the original function $\mathcal{f}$. At least in theory, an arbitrarily accurate approximation should be attainable by increasing the degree ($\mathcal{n}$) of approximation.   
      Second, it should be possible to compute the basis coeffcients quickly and accurately $\rightarrow$ the interpolation equation should be well-conditioned and should be easy to solve $\rightarrow$ diagonal, near diagonal, or orthogonal interpolation matrices ($\Phi$) are best.      
      Third, the approximant should be easy to work with: the basis functions should be easy and relatively costless to evaluate, differentiate, and integrate.   
      
Interpolation is a collection of methods to approximate a function. We distinguish between the ***spectral*** methods and ***finite element methods***. ***Spectral*** interpolation schemes use basis functions that are nonzero over the entire domain of $\mathcal{f}$, e.g. polynomial interpolation. ***Finite element methods*** use basis functions that are nonzero only over a subinterval of the domain of  $\mathcal{f}$, e.g. splines. 

## Polynomial interpolation      

According to the Weierstrass Theorem, any continuous real-valued function f defined on a bounded interval [a,b] of the real line can be approximated to any degree of accuracy using a polynomial. The Theorem only confirms the existence of a polynomial to approximate $\mathcal{f}$ up to any degree of accuracy, but gives no guidance on how the approximation is constructed. Developing a polynomial interpolation scheme, however, entails two main questions: choice of the ***basis functions*** and choice of the ***interpolation nodes***    

Intuitively, we would chose ***monomial basis polynomial***     
$$p(x) = c_1x^0 + c_2 x + c_3x^2 + ... +c_n x^{n-1}$$   
that interpolates $\mathcal{f}$ at *n* evenly spaced (uniformly distributed) ***interpolation nodes***    
    $$x_i = a + \frac{i-1}{n-1}(b-a)   \forall i = 1,2,...,n $$

In [13]:
import numpy as np

In [15]:
x = sym.Symbol('x')
f = 10*(x-1)**4 - 1
psi = [1, x]
Omega = [0,1]
points = np.linspace(0,1,2)
c = interpolation(f, psi, points)  

In [16]:
c

Matrix([
[  9.0],
[-10.0]])

Problem: we don't ncrease approximation precision by increasing the order of interpolation.    
Solution: use ***Chebyshev basis functions*** and ***Chebyshev interpolation nodes*** instead. 